<strong>This notebook attempts to answer the following questions below:</strong>

# NASA Satellite Data Analysis
John Bonfardeci<br/>
2021-01-31

# Questions
1. Total Number of Satellites.
2. Total Active Satellites.
3. Number of unique vessels.
4. Number of estimated fishing boats.
5. Number of vessels missing names or voyage info.
6. Number of satellites missing names.
7. Number of valid voyages within the Bering Sea area, number outside the Bering Sea.
8. Number of vessels that "seam" to have satellite tracking capability, i.e. voyages away from shore: 
9. Number of vessels outside the 48 mile coastline - See #8

Apache Spark and access to the AWS Vault S3 bucket, `af-vault` is required.

In [31]:
# Import Dependencies
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as F
import numpy as np

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
sql = SQLContext(spark.sparkContext)

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('DataQuestions') \
    .getOrCreate()

## Import AIS Data: Broadcast, Voyage, & Vessel Tables

In [3]:
ais_folder = 's3://af-vault/Raw/AIS/extracted/'
broadcast = spark.read.csv(ais_folder+'/*.Broadcast.csv', header=True)
vessel = spark.read.csv(ais_folder+'/*.Vessel.csv', header=True)
voyage = spark.read.csv(ais_folder+'/*.Voyage.csv', header=True)
sql.registerDataFrameAsTable(broadcast, 'broadcast')
sql.registerDataFrameAsTable(vessel, 'vessel')
sql.registerDataFrameAsTable(voyage, 'voyage')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
broadcast.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---+---+-------+---+-------------------+------+--------+---------+------------+----------+
|            geometry|SOG|COG|Heading|ROT|       BaseDateTime|Status|VoyageID|     MMSI|ReceiverType|ReceiverID|
+--------------------+---+---+-------+---+-------------------+------+--------+---------+------------+----------+
|POINT (-122.71500...|  0|199|    511|128|2008-12-31T23:58:59|     0|       1|367047170|           r|   13SCDS1|
|POINT (-122.86996...|  0|325|    511|128|2008-12-31T23:58:59|     0|       2|366763770|           r|   13SCDS1|
|POINT (-122.33062...|  0|354|    511|128|2008-12-31T23:58:59|     0|       3|368494000|           r|   13RSMT1|
|POINT (-122.64572...|  0|345|     55|  0|2008-12-31T23:58:59|     0|       4|366116000|           b| 003669700|
|POINT (-123.03351...|  0|172|     96|  0|2008-12-31T23:58:59|    15|       5|316003289|           b| 003669705|
+--------------------+---+---+-------+---+-------------------+------+--------+---------+--------

In [34]:
vessel.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+-------+--------+------------------+----------+------+-----+-------------------+
|geometry|     MMSI|    IMO|CallSign|              Name|VesselType|Length|Width|DimensionComponents|
+--------+---------+-------+--------+------------------+----------+------+-----+-------------------+
|    null|367061930|      0|  WX8482|          ROBERT S|         0|    26|    8|           21,5,2,6|
|    null|316003349|      0| CFG7268|        CALEDONIAN|        30|    30|    8|          12,18,2,6|
|    null|305255000|9366110|   V2DM2|       BBC ORINOCO|        70|   141|   22|       17,124,10,12|
|    null|366659730|      0| WCN2606|CHRISTINE ANDERSON|         0|     0|    0|            0,0,0,0|
|    null|366986380|      0| WUV6842|     DREDGE OREGON|        33|    61|   18|           9,52,9,9|
+--------+---------+-------+--------+------------------+----------+------+-----+-------------------+
only showing top 5 rows

In [35]:
voyage.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+---------------+-----+-------+-------------------+-------------------+-------------------+---------+
|geometry|VoyageID|    Destination|Cargo|Draught|                ETA|          StartTime|            EndTime|     MMSI|
+--------+--------+---------------+-----+-------+-------------------+-------------------+-------------------+---------+
|    null|     109|      TENDERING|    0|      0|               null|2008-12-31T23:59:55|2009-01-14T01:56:57|367061930|
|    null|     346|SFO ANCHORAGE 9|   70|     65|2008-01-01T02:00:00|2008-12-31T23:59:55|2009-01-01T11:41:32|305255000|
|    null|     261|      SKAMOKAWA|   33|     40|2009-09-08T12:00:00|2009-01-01T00:00:01|               null|366986380|
|    null|     451|     BELLINGHAM|   70|     92|2008-07-17T13:30:00|2008-12-31T23:59:58|               null|366604000|
|    null|       9|         RCH 11|   52|     40|2009-12-30T13:30:00|2009-01-01T00:00:00|2009-01-25T13:14:09|366763440|
+--------+--------+---------------+-----

## Fishing Vessels

In [36]:
fishing_wessels = sql.sql("""
SELECT count(distinct MMSI) as uniques FROM vessel WHERE cast(VesselType as int) = 30 
""")
fishing_wessels.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|uniques|
+-------+
|    431|
+-------+

## Unique Ship IDS (MMSI)

In [37]:
# Unique Vessels
unique_ships = sql.sql("""
SELECT count(distinct MMSI) as uniques FROM vessel
""")
unique_ships.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|uniques|
+-------+
|   7179|
+-------+

## Missing Names
Joined to Sat-Cat dataset on NORAD ID.<br/>
https://celestrak.com/satcat/search.php

In [38]:
missing_names = sql.sql("""
SELECT count(distinct MMSI) as uniques FROM vessel WHERE Name IS NULL
""")
missing_names.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|uniques|
+-------+
|   5454|
+-------+

## Missing Voyage Data

In [39]:
missing_voyage_data = sql.sql("""
SELECT 
    count(distinct v.MMSI) as uniques
FROM
    vessel v
    LEFT OUTER JOIN voyage vg ON v.MMSI = vg.MMSI
WHERE
    vg.MMSI IS NULL
""")
missing_voyage_data.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|uniques|
+-------+
|    977|
+-------+

## TLE Satellite Data

In [40]:
tle = spark.read.csv('s3://af-vault/Raw/TLE/cleaned/*.txt', sep='|', )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
tle.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------+--------------------+
|  _c0|                 _c1|                 _c2|                 _c3|
+-----+--------------------+--------------------+--------------------+
|28082|2018-01-01T23:09:...|1 28082U 03052A  ...|2 28082   3.8908 ...|
|39613|2018-01-01T20:42:...|1 39613U 14010B  ...|2 39613   0.0570 ...|
|39487|2018-01-01T20:42:...|1 39487U 13077A  ...|2 39487   0.0598 ...|
|28909|2018-01-01T21:29:...|1 28909U 05048B  ...|2 28909  82.4696 ...|
|31698|2018-01-01T23:36:...|1 31698U 07026A  ...|2 31698  97.4442 ...|
+-----+--------------------+--------------------+--------------------+
only showing top 5 rows

In [42]:
sql.registerDataFrameAsTable(tle, 'tle')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Distinct Satellite IDs

In [43]:
unique_sats = sql.sql("SELECT count(distinct _c0) as Uniques FROM tle")
unique_sats.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|Uniques|
+-------+
|  23500|
+-------+

## Import Sat Cat Dataset

In [44]:
satcat = spark.read.csv('s3://af-vault/Raw/TLE/lookups/satcat.csv', header=True)
satcat.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+------------+-----------+---------------+-----+-----------+-----------+----------+------+-----------+------+-------+-------+----------------+------------+----------+
|OBJECT_NAME|OBJECT_ID|NORAD_CAT_ID|OBJECT_TYPE|OPS_STATUS_CODE|OWNER|LAUNCH_DATE|LAUNCH_SITE|DECAY_DATE|PERIOD|INCLINATION|APOGEE|PERIGEE|    RCS|DATA_STATUS_CODE|ORBIT_CENTER|ORBIT_TYPE|
+-----------+---------+------------+-----------+---------------+-----+-----------+-----------+----------+------+-----------+------+-------+-------+----------------+------------+----------+
|   SL-1 R/B|1957-001A|           1|        R/B|              D|  CIS| 1957-10-04|      TYMSC|1957-12-01| 96.19|      65.10|   938|    214|20.4200|            null|          EA|       IMP|
|  SPUTNIK 1|1957-001B|           2|        PAY|              D|  CIS| 1957-10-04|      TYMSC|1958-01-03| 96.10|      65.00|  1080|     64|   null|            null|          EA|       IMP|
+-----------+---------+------------+-----------+-------

In [45]:
sql.registerDataFrameAsTable(satcat, 'satcat')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Active Satellites in Sat Cat

In [46]:
active = sql.sql("""
SELECT 
    count(distinct OBJECT_NAME) as ActiveCount
FROM 
    satcat
WHERE 
    OPS_STATUS_CODE IN ('+', 'P', 'B', 'S', 'X')
""")
active.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|ActiveCount|
+-----------+
|       3519|
+-----------+

In [47]:
active.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

## Get Satellite Names

In [48]:
sat_names = sql.sql("""
SELECT 
    _c0 as SatNum
    , OBJECT_NAME as SatName
    , CASE WHEN OPS_STATUS_CODE IN ('+', 'P', 'B', 'S', 'X') THEN 1 
    ELSE 0 END as Operational
FROM 
    tle 
    LEFT OUTER JOIN satcat ON tle._c0 = satcat.NORAD_CAT_ID
GROUP BY 
    _c0, OBJECT_NAME, OPS_STATUS_CODE
""")
sat_names.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------------+-----------+
|SatNum|            SatName|Operational|
+------+-------------------+-----------+
| 10840|        DELTA 1 DEB|          0|
| 21386|        DELTA 1 DEB|          0|
| 06660|               null|          0|
| 34204|        CBERS 1 DEB|          0|
| 35622|     IRIDIUM 33 DEB|          0|
| 12904|           SL-3 R/B|          0|
|  1739|      SCOUT X-4 R/B|          0|
| 17107|    COSMOS 1375 DEB|          0|
| 22375|          SL-16 DEB|          0|
| 23482|          SL-19 DEB|          0|
| 26654|          CZ-4B DEB|          0|
| 26829|        DELTA 1 DEB|          0|
| 35227|     FENGYUN 1C DEB|          0|
| 27601|           H-2A R/B|          0|
| 41581|INTELSAT 31 (IS-31)|          1|
| 30716|     FENGYUN 1C DEB|          0|
| 38087|              SES-4|          1|
| 35102|     FENGYUN 1C DEB|          0|
|   608|              ERS 6|          0|
| 40081|            O3B FM6|          1|
+------+-------------------+-----------+
only showing top

In [49]:
sql.registerDataFrameAsTable(sat_names, 'satnames')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Active Satellites in TLE Data - Joined to Sat Cat

In [50]:
active = sql.sql("SELECT count(distinct SatNum) NumActive FROM satnames WHERE Operational = 1")
active.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|NumActive|
+---------+
|     1776|
+---------+

## Missing Satellite Names - Joined to Sat Cat

In [58]:
missing_names = sql.sql("SELECT count(distinct SatNum) MissingNames FROM satnames WHERE SatName IS NULL")
missing_names.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|MissingNames|
+------------+
|        2466|
+------------+

## Ships within Bering Sea Area

In [30]:
from math import radians, sin, cos, sqrt, asin

def get_distance(lat_a, lon_a, lat_b, lon_b): 
    lon1,  lat1, lon2, lat2 = map(radians, [lat_a, lon_a, lat_b, lon_b])
    
    a = sin((lat2-lat1)/2.0)**2 + \
        cos(lat1) * cos(lat2) * sin((lon2-lon1)/2.0)**2
    
    earth_radius = 3798
    return earth_radius * 2 * asin(sqrt(a))

def bering_sea_dist(lat, lon):
    # Approx center of Bering Sea
    coords = [59.07926040162669, -178.2428626370946]
    return get_distance(lat, lon, coords[0], coords[1])

udf_get_distance = F.udf(bering_sea_dist)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
geom = sql.sql("""
SELECT 
    MMSI
    , BaseDateTime 
    , cast(trim(geom[0]) as Double) as lat
    , cast(trim(geom[1]) as Double) as lon
    , 0 as DistanceToBeringSeaCenter
FROM (
    SELECT 
        MMSI
        , BaseDateTime
        , split(replace(replace(geometry, 'POINT (', ''), ')', ''), ' ') as geom 
    FROM 
        broadcast 
) t
""")
geom.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-----------+-----------------+-------------------------+
|     MMSI|       BaseDateTime|        lat|              lon|DistanceToBeringSeaCenter|
+---------+-------------------+-----------+-----------------+-------------------------+
|367047170|2008-12-31T23:58:59|-122.715003|45.63362699999999|                        0|
|366763770|2008-12-31T23:58:59|-122.869968|        46.027173|                        0|
|368494000|2008-12-31T23:58:59|-122.330622|47.54250200000001|                        0|
|366116000|2008-12-31T23:58:59|-122.645723|48.51261199999999|                        0|
|316003289|2008-12-31T23:58:59|-123.033517|49.14422500000001|                        0|
|431162000|2008-12-31T23:58:59|-123.714967|        34.575867|                        0|
|367029760|2008-12-31T23:58:59| -122.49624|48.39882700000001|                        0|
|366862680|2008-12-31T23:58:59|-122.214065|37.50421799999999|                        0|
|366763440|2008-12-31T23:58:59| 

In [23]:
dist_bering = geom.withColumn('DistanceToBeringSeaCenter'
                                , udf_get_distance(geom.lat, geom.lon))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
dist_bering.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-----------+-----------------+-------------------------+
|     MMSI|       BaseDateTime|        lat|              lon|DistanceToBeringSeaCenter|
+---------+-------------------+-----------+-----------------+-------------------------+
|367047170|2008-12-31T23:58:59|-122.715003|45.63362699999999|        3143.188248327428|
|366763770|2008-12-31T23:58:59|-122.869968|        46.027173|        3169.566290929054|
|368494000|2008-12-31T23:58:59|-122.330622|47.54250200000001|        3268.978211213409|
|366116000|2008-12-31T23:58:59|-122.645723|48.51261199999999|        3333.742909730551|
|316003289|2008-12-31T23:58:59|-123.033517|49.14422500000001|       3376.3041988819086|
|431162000|2008-12-31T23:58:59|-123.714967|        34.575867|        2414.693293885854|
|367029760|2008-12-31T23:58:59| -122.49624|48.39882700000001|       3325.9595843650795|
|366862680|2008-12-31T23:58:59|-122.214065|37.50421799999999|       2603.7532729843124|
|366763440|2008-12-31T23:58:59| 

In [25]:
dist_bering.orderBy('DistanceToBeringSeaCenter').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-----------+-----------------+-------------------------+
|     MMSI|       BaseDateTime|        lat|              lon|DistanceToBeringSeaCenter|
+---------+-------------------+-----------+-----------------+-------------------------+
|857479069|2014-01-04T07:17:46|-121.854787|13.35459299999999|       1003.6012693882986|
|308662000|2009-12-29T17:05:00|-123.939948|        13.631152|       1039.1762913762122|
|563745791|2014-01-14T13:53:34|  -124.0257|         13.92913|       1059.6270275475924|
|367561462|2014-01-26T23:49:05| -122.72039|        14.178828|       1062.9490258158537|
|692873301|2010-01-09T00:11:00|-122.514362|14.26399000000001|       1067.1461452926585|
|440002508|2011-01-15T09:24:00|-125.090428|        13.807115|       1067.4798966269007|
|533390100|2014-01-03T04:46:40|-120.653983|14.39231700000001|       1070.6519228996901|
|533390100|2014-01-03T04:51:40|  -120.6748|         14.39395|       1070.7386703608852|
|533390100|2014-01-03T04:53:14|-

## Voyages Outside the Bering Sea
Voyages outside a radius of ~ 1100km

In [29]:
dist_bering.where('DistanceToBeringSeaCenter > 1100').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

121853046

## Voyages Inside the Bering Sea
Voyages within a radius of ~ 1100km

In [ ]:
dist_bering.where('DistanceToBeringSeaCenter <= 1100').count()

Approx center of Bering Sea area: lat:59.07926040162669, lon:-178.2428626370946<br/>
Approx distance from center to perimiter: 1100 km

In [27]:
# test
test_dist = bering_sea_dist(57.7227961971243, 164.1471189278966) # 1170.7038386396125
print("Distance to center of Bering Sea %f" % (test_dist))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Distance to center of Bering Sea 1170.703839

## Answers

1. Total Number of Satellites: 23,500
2. Total Active Satellites: 1,776 per external data (as of 1/5/2021, https://celestrak.com/satcat/search.php)
3. Number of unique vessels: 7,179
4. Number of estimated fishing boats: 431 where VesselType = 30 (Fishing)
5. Number of vessels missing names or voyage info: 5,454 and 977
6. Number of satellites missing names: 2,466
7. Number of valid voyages within the Bering Sea area, number outside the Bering Sea: 127 and 121,853,046 respectively.
8. Number of vessels that "seam" to have satellite tracking capability, i.e. voyages away from shore: 
    - Need data set of lat/lon to define all shorelines
    - Use Haversine to calculate distance to shore for each ship for at least each hour of its voyage. 
    - Data gathering and compute time is a limitation given time frame.
9. Number of vessels outside the 48 mile coastline - See #8

* Satellite operational status according to ('+', 'P', 'B', 'S', 'X')